<a href="https://colab.research.google.com/github/RITIKA-01A/sentiment_analysis_LSTM/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("hellow world")

hellow world


In [2]:
!pip install kaggle

In [3]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Embedding , LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


**Data collection - kaggle api**

In [4]:
kaggle_dict = json.load(open("kaggle.json"))

In [5]:
#setup kaggle credentials as environment variable
os.environ["KAGGLE_USERNAME"] = kaggle_dict["username"]
os.environ["KAGGLE_KEY"] = kaggle_dict["key"]


In [6]:
!

In [7]:
!lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

/bin/bash: line 1: lakshmi25npathi/imdb-dataset-of-50k-movie-reviews: No such file or directory


In [8]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other


In [9]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [10]:
#unzip the dataset file
with ZipFile("/content/imdb-dataset-of-50k-movie-reviews.zip" , "r") as zip_ref:
  zip_ref.extractall()



In [11]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


**Loading the data**

In [12]:
df = pd.read_csv("IMDB Dataset.csv")

In [13]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [14]:
df.shape

(50000, 2)

In [15]:
df['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [16]:
df.replace({"sentiment":{"positive":1 , "negative":0}} , inplace=True)

<ipython-input-16-4742407ca5cc>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({"sentiment":{"positive":1 , "negative":0}} , inplace=True)


In [17]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [18]:
df['sentiment'].value_counts()

,count
sentiment,
1,25000
0,25000


In [19]:
## spiltting the data into trai  and test
train_data , test_data = train_test_split(df , test_size=0.2 , random_state=42)

In [20]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


**Data Preprocessing**

In [21]:
##tokenize
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]) , maxlen = 200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]) , maxlen = 200)

In [22]:
 print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [23]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [24]:
y_train = train_data["sentiment"]
y_test = test_data["sentiment"]

In [25]:
print(y_train.shape)

(40000,)


**BUILDING THE LSTM MODEL**

In [26]:
model = Sequential()
model.add(Embedding(input_dim = 5000 , output_dim = 128 , input_length = 200))
model.add(LSTM(128 , dropout=0.2 , recurrent_dropout=0.2))
model.add(Dense(1 , activation="sigmoid"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [27]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [28]:
model.compile(optimizer="adam" , loss = "binary_crossentropy" ,metrics=["accuracy"])

In [29]:
model.fit(X_train , y_train , epochs = 5 , batch_size=64 , validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 295s 581ms/step - accuracy: 0.7355 - loss: 0.5192 - val_accuracy: 0.7811 - val_loss: 0.4600
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 324s 586ms/step - accuracy: 0.8572 - loss: 0.3453 - val_accuracy: 0.8641 - val_loss: 0.3250
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 319s 580ms/step - accuracy: 0.8808 - loss: 0.2992 - val_accuracy: 0.8584 - val_loss: 0.3417
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 325s 586ms/step - accuracy: 0.8654 - loss: 0.3228 - val_accuracy: 0.8699 - val_loss: 0.3145
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 321s 585ms/step - accuracy: 0.9063 - loss: 0.2423 - val_accuracy: 0.8771 - val_loss: 0.3056


**MODEL EVALUATION**

In [30]:
loss , accurracy = model.evaluate(X_test , y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 27s 85ms/step - accuracy: 0.8831 - loss: 0.2919


In [34]:
print("loss :" ,loss )
print("accuracy : " , accurracy)

loss : 0.2899027466773987
accuracy :  0.8840000033378601


**BUILDING A PREDICTIVE SYSTEM**

In [40]:
def predict_sentiment(review):
  ##toeknize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence , maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment



In [41]:
#example usage
new_review = "the movie was very very bad i just hated it"
predict_sentiment(new_review)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step


'negative'

In [42]:
#example usage
new_review = "the movie was so good. i just loved it"
predict_sentiment(new_review)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step


'positive'